# Découverte de la base de donnée sur les incidents
## Chargement de la base et visualisation de quelques lignes

In [310]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
url_api="https://ressources.data.sncf.com/api/v2/catalog/datasets/incidents-securite/exports/json?limit=-1&offset=0&lang=fr&timezone=Europe%2FParis"
incident= pd.read_json(url_api)
incident.head()

,id,origine,numero_isic,type_event,date,nature,region,lieu,niveau_gravite
0,18212,Mobilités,0,Dépassement de la vitesse limite de plus de 40...,2015-01-12,Dépassement de vitesse supérieur à 40 km/h,RA,Montbrison (42),NaN
1,18152,Réseau,None,RFC,2015-01-14,"Un TGV franchi le PN 25 barrières hautes, la s...",None,Lunel (34),NaN
2,18196,Réseau,None,Défaillance voie,2015-01-19,Défaut de géométrie nécessitant l'arrêt des ci...,LR,Entre Sète et Le Castellas,NaN
3,18203,Mobilités,17347,Franchissement intempestif d'un signal d'arrêt...,2015-01-20,Le signal C348 est franchit fermé par le train...,CEN,Orléans,NaN
4,18379,Réseau,17477,Engagement intempestif d'un train sur une voie...,2015-02-04,Engagement intempestif d'une circulation sur u...,RA,Lyon Perrache (69),NaN


## Description

Ce jeu de données porte sur les incidents de sécurité survenus sur le réseau de la SNCF depuis janvier 2015 jusqu'au 28 octobre 2022. Il contient 1651 observations qui sont enrégistrées mensuellement. Ne sont listés dans ce fichier que les événements de sécurité impliquant un dysfonctionnement du système ferroviaire, qu'il soit d'origine interne ou externe. Il contient 9 variables à savoir:
* *id et numero_isic*: pour l'identifiant des incidents
* *origine*: 
* *type_event*: elle renseigne sur le type d'évenement survenu,
* *date*: elle renseigne sur la date de survenue de l'événement
* *nature*: elle renseigne sur la nature de l'événement
* *region*: elle renseigne sur la région dans laquelle est survenu l'événement
* *lieu*: elle renseigne sur le lieu où est survenu l'événement
* *niveau_gravite*: elle renseigne sur le niveau de gravité de l'incident. L'échelle de gravité couvre 6 niveaux, le niveau 1 correspond à un événement "mineur"  et le niveau 6 correspond à un évènement qui a eu des conséquences graves. 

## Données manquantes
Nous nous intéressons dans cette partie aux données manquantes sur les différentes variables exceptées les variables id et numero_isic qui ne présentent aucun intérêt pour notre étude.

In [311]:
df1 = incident.drop(columns=['numero_isic', 'id'])   # on retire l'id et numero_isic
df1= df1[df1.isnull().any(axis=1)] # filtrage aux observations comportant des données manquantes
len(df1)

482

Au total, 482 incidents ont été enrégistrés avec des informations manquantes sur l'une au moins des variables. 

Regardons les données manquantes par variable.


In [312]:
df1.isnull().sum()

origine            19
type_event         20
date               19
nature             10
region             61
lieu               20
niveau_gravite    466
dtype: int64

Exceptée la variable *niveau_gravite* dont environ 28% de ces données sont manquantes, les autre variables ont une proprtion relativement faible de données manquantes.

### Mais d'où viennent toutes ces données manquantes sur le niveau de gravite?

In [313]:
incident['annee']=incident['date'].dt.strftime('%Y')
def miss_by_year(variable): #cette fonction retourne un tableau sur le nombre de valeurs manquantes par année de la variable de la base "incident" passé en argument
    annee1=np.array(incident['annee'].unique())
    annee1=np.delete(annee1,1)
    count_na=[]
    for i in annee1:
        df=incident[incident['annee']==i]
        count_na.append(df[variable].isnull().sum())
    count_na=np.array(count_na)
    df2=incident.groupby('annee').agg({'annee': "count"})
    tab=pd.DataFrame({"Annee":annee1, "Nbre_valeurs_manquantes":count_na, "Nbre_enregistrements":df2['annee']})
    tab=tab.reset_index(drop=True)
    return(tab)
 
miss_by_year("niveau_gravite")

,Annee,Nbre_valeurs_manquantes,Nbre_enregistrements
0,2015,308,308
1,2016,0,242
2,2017,2,225
3,2018,0,217
4,2020,139,164
5,2021,3,184
6,2019,1,157
7,2022,0,135


Sur les 308 incidents enrégistrés en 2015, auncune information sur le niveau de gravité de ces incidents n'a été enrégistrée. Ces données manquantes sur le niveau de gravité des incidents en 2015 représentent environ 66% du total des données manquantes sur cette variable sur l'ensemble des données.

On peut déduire qu'en 2015, l'indicateur mesurant le niveau de gravité des incidents ie "niveau_gravite" n'avait pas encore été mis en place. On choisit de ne pas imputer les données manquantes sur le niveau de gravité en 2015 puisqu'elles n'ont jamais existé. Dans le traitement des données, on ignorera les données de 2015 dans les cas où l'analyse implique la variable niveau_gravite.

## Traitement des données manquantes

In [314]:
#supression des valeurs extrêmes
incident=incident.drop(incident[(incident['niveau_gravite']<1)  | (incident['niveau_gravite']>6)].index, inplace=False)

Nous procédons dans cette partie aux traitements des valeurs manquantes par variable. Pour cela, nous avons implenté une fonction `nettoyage` qui prend en paramètre le nom de la variable dont on veut traiter les valeurs manquantes.

### Description de la fonction

Pour une ligne i donnée ayant une valeur manquante pour la variable en paramètre, l'idée est de regrouper toutes les lignes n'ayant pas de valeur pas de valeur manquante pour la variable à imputer et qui ressemblent plus à la ligne i du point de vue de trois facteurs que sont: l'année (`annee`), le type d'évenement (`type_event`) et l'origine (`origine`). On obtient alors une classe de ressemblance pour la ligne i qu'on utilise ensuite pour l'imputation. Si la variable à imputer (donc en argument) de la fonction est l'un de nos 03 facteurs , on ignore ce facteur dans la constitution de la classe de ressemblance. De même, lorsque la valeur de la ligne i pour l'un des facteurs est manquante, on ignore ce facteur dans la constitution de la classe de ressemblance. Par ailleurs, dans la constitution de la classe de ressemblance, les facteurs sont sélectionnés successivement dans l'ordre: annee, type_event, origine. Lorsqu'en ajoutant un facteur donné, la classe devient vide, cela signifie qu'il n'existe aucune obervation (ligne) ayant les mêmes valeurs que la ligne i pour les facteurs sélectionnés. Le dernier facteur ajouté sera alors ignoré dans la constitution de la classe d'équivalence. Une fois la classe construite, on impute la valeur manquante par la moyenne ou par le mode de la variable à imputer selon le type de cette variable.

In [315]:
def nettoyage(var):
    if var=="niveau_gravite":
        year=["2016","2017","2018","2019","2020","2021","2022"]
    if var!="niveau_gravite":
        year=["2015","2016","2017","2018","2019","2020","2021","2022"]
    factor=["type_event", "origine"]
    for i in year:
        data=incident.loc[(incident[var].isnull()==True) & (incident['annee']==i)]
        if len(data)!=0:
            index=data.index
            for j in range(len(index)):
                df=incident.loc[(incident['annee']==i) & (incident[var].isnull()==False)]
                if var!="type_event":
                    if (len(df.loc[incident['type_event']==incident['type_event'][index[j]]])!=0) & (incident['type_event'][index[j]]!=np.nan):
                        df=df.loc[(incident['type_event']==incident['type_event'][index[j]])]
                if var!="origine":
                    if (len(df.loc[(incident['origine']==incident['origine'][index[j]])])!=0) & (incident['origine'][index[j]]!=np.nan):
                        df=df.loc[(incident['origine']==incident['origine'][index[j]])]
                if var=="niveau_gravite":
                    incident[var][index[j]]=df[var].mean()
                if var!="niveau_gravite":
                    incident[var][index[j]]=df[var].mode()[0]
    return(incident)


#### Traitement des valeurs manquantes de la variable `niveau_gravite`
On utilise la fonction `nettoyage` pour imputer les valeurs manquantes de la variable `niveau_gravite`. Comme indiqué ci-dessus, l'année 2015 ne sera pas considérée pour cette variable. 

In [316]:
incident=nettoyage("niveau_gravite")           
miss_by_year("niveau_gravite")

,Annee,Nbre_valeurs_manquantes,Nbre_enregistrements
0,2015,308,308
1,2016,0,242
2,2017,0,225
3,2018,0,217
4,2020,0,164
5,2021,0,184
6,2019,0,157
7,2022,0,135


#### Traitement des valeurs manquantes de la variable `annee`


In [317]:
incident.loc[incident['date'].isnull()==True].head()

,id,origine,numero_isic,type_event,date,nature,region,lieu,niveau_gravite,annee
66,None,None,None,None,NaT,Blanc = ESR Autre,None,None,NaN,NaN
346,None,None,None,None,NaT,Marron = ESR NG Infra,None,None,NaN,NaN
347,None,None,None,None,NaT,Vert Foncé = ESR NG Mobilités,None,None,NaN,NaN
394,None,None,None,None,NaT,None,None,None,3.785124,NaN
435,None,None,None,None,NaT,Marron = ESR Infra M et T,None,None,NaN,NaN


On note que lorsque l'année d'une obervation n'a pas été renseignée, aucune des autres variables n'est renseignée sauf peut-être la nature. Il s'agit d'une non-réponse totale si la nature n'est pas considérée.

On décide de supprimer les observations dont l'année n'a pas été renseignée.

In [318]:
incident=incident.drop(incident[incident['date'].isnull()==True].index, inplace=False)
df1 = incident.drop(columns=['numero_isic', 'id'])
df1.isnull().sum()

origine             0
type_event          1
date                0
nature              2
region             43
lieu                1
niveau_gravite    308
annee               0
dtype: int64

#### Traitement des valeurs manquantes de la variable `type_event`

In [319]:
nettoyage("type_event")
df1 = incident.drop(columns=['numero_isic', 'id'])
df1.isnull().sum()

origine             0
type_event          0
date                0
nature              2
region             43
lieu                1
niveau_gravite    308
annee               0
dtype: int64

#### Traitement des valeurs manquantes de la variable `region`

In [320]:
incident=nettoyage("region")
df1 = incident.drop(columns=['numero_isic', 'id'])
df1.isnull().sum()

origine             0
type_event          0
date                0
nature              2
region              0
lieu                1
niveau_gravite    308
annee               0
dtype: int64

Les variables `nature` et `lieu` ne seront pas imputées parce qu'ayant presque autant de modalités que d'observations

In [321]:
len(incident['nature'].unique())

1601

In [322]:
len(incident['lieu'].unique())

1300

On enregistre la nouvelle base obtenue qui sera ensuite utilisée pour la visualisation.

In [324]:
incident.to_csv('incident.csv', index=False)